# Computational sectioning

In [ ]:
#!python -m pip install numpy pandas scikit-image simutome tifffile

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
from simutome.estim import CellSlicer
from skimage.morphology import binary_erosion, disk
from tifffile import imread, imwrite

In [ ]:
img_file = "sectioning/img.tiff"
mask_file = "sectioning/mask.tiff"
panel_file = "sectioning/panel.csv"
voxel_size_um = (2.0, 1.0, 1.0)

sectioning_axes = [2, 1, 0]
section_thicknesses_um = [2.0, 4.0, 6.0, 8.0, 10.0]
min_proj_cell_slice_area_um2 = 5.0

cell_data_file = "sectioning/cell_data.csv"
cell_slice_data_file = "sectioning/cell_slice_data.csv"
cell_slice_displacement_data_file = "sectioning/cell_slice_displacement_data.csv"

## Data preparation

In [ ]:
# sort layers
layer_info = pd.read_csv(
    "raw/MainHer2BreastCancerModel/final_3D_stack_order_model201710.csv",
    names=["image", "order"],
)
img_file_stems = layer_info.sort_values("order")["image"].tolist()

# sort channels
channel_dirs = sorted(
    Path("raw/MainHer2BreastCancerModel/SIMILARITY10_Nd148").iterdir(),
    key=lambda p: int(p.name[2:]),
)

# cropping (see raw/MainHer2BreastCancerModel/roi_overlapping_coordinates)
img_slice = np.s_[448:936, 386:1038]
orig_panel = pd.read_csv("raw/MainHer2BreastCancerModel/model201710_panel.csv")
channel_mask = orig_panel.loc[
    orig_panel["singleTIFFs"] == 1,
    "MeasureMask"
].to_numpy() == 1

# read image
layer_imgs = []
for i, img_file_stem in enumerate(img_file_stems):
    channel_imgs = []
    for channel_dir in channel_dirs:
        channel_img = imread(
            channel_dir / f"{i + 1:04d}_{img_file_stem}_{channel_dir.name}.tif"
        )[img_slice]
        channel_imgs.append(channel_img)
    layer_img = np.stack(channel_imgs)[channel_mask, :, :]
    layer_imgs.append(layer_img)
img = np.stack(layer_imgs)

In [ ]:
# read mask
mask = imread(
    "raw/MainHer2BreastCancerModel/measured_mask_final_segmentation_hwatershed_500.00_90%.tif"
)

# erode previously dilated mask
mask *= binary_erosion(mask != 0, disk(1.0)[None, :, :])

# remove border cells to avoid biases
border_cells = np.unique(
    np.concatenate(
        (
            np.unique(mask[[0, -1], :, :]),
            np.unique(mask[:, [0, -1], :]),
            np.unique(mask[:, :, [0, -1]]),
        )
    )
)
mask[np.isin(mask, border_cells)] = 0

assert mask.ndim == 3
assert mask.shape[0] == img.shape[0]
assert mask.shape[1:] == img.shape[2:]

In [ ]:
# crop image & mask to speed up computation
ix = np.flatnonzero(np.amax(mask, axis=(0, 1)) != 0)
iy = np.flatnonzero(np.amax(mask, axis=(0, 2)) != 0)
iz = np.flatnonzero(np.amax(mask, axis=(1, 2)) != 0)
img = img[:, :, :, np.amin(ix):np.amax(ix) + 1]
img = img[:, :, np.amin(iy):np.amax(iy) + 1, :]
img = img[np.amin(iz):np.amax(iz) + 1, :, :, :]
mask = mask[:, :, np.amin(ix):np.amax(ix) + 1]
mask = mask[:, np.amin(iy):np.amax(iy) + 1, :]
mask = mask[np.amin(iz):np.amax(iz) + 1, :, :]

In [ ]:
# read panel
orig_panel = pd.read_csv("raw/MainHer2BreastCancerModel/model201710_panel.csv")
orig_panel = orig_panel[orig_panel["MeasureMask"] == 1]
assert len(orig_panel.index) == img.shape[1]
channel_names = orig_panel["clean_target"].tolist()
channel_labels = orig_panel["Target"].tolist()
panel = pd.DataFrame(
    data={
        "name": channel_names,
        "label": channel_labels,
    }
)
assert len(panel.index) == img.shape[1]

In [ ]:
# write data
Path("sectioning").mkdir(exist_ok=True)
imwrite(img_file, data=img)
imwrite(mask_file, data=mask)
panel.to_csv(panel_file, index=False)

## Computational sectioning

In [ ]:
img = imread(img_file)
mask = imread(mask_file)
panel = pd.read_csv(panel_file)
cell_slicer = CellSlicer(
    mask,
    image=img,
    channel_names=panel["name"].tolist(),
    voxel_size_um=voxel_size_um,
)

In [ ]:
cell_data = cell_slicer.measure_cells(sectioning_axes, progress=True)
cell_data.insert(
    1,
    "sectioning_axis_name",
    np.array(["z", "y", "x"])[cell_data["sectioning_axis"].values],
)
cell_data["cell_radius_um"] = (0.75 * cell_data["cell_volume_um3"] / np.pi) ** (1 / 3)
cell_data["proj_cell_radius_um"] = (cell_data["proj_cell_area_um2"] / np.pi) ** 0.5
cell_data.to_csv(cell_data_file, index=False)

In [ ]:
for i, sectioning_axis in enumerate(sectioning_axes):    
    cell_slice_data = cell_slicer.measure_cell_slices(
        [sectioning_axis], section_thicknesses_um, progress=True
    )
    cell_slice_data = cell_slice_data[
        cell_slice_data["proj_cell_slice_area_um2"] >= min_proj_cell_slice_area_um2
    ]
    cell_slice_data.insert(
        1,
        "sectioning_axis_name",
        np.array(["z", "y", "x"])[cell_slice_data["sectioning_axis"].values],
    )
    cell_slice_data["cell_slice_radius_um"] = (
        0.75 * cell_slice_data["cell_slice_volume_um3"] / np.pi
    ) ** (1.0 / 3.0)
    cell_slice_data["proj_cell_slice_radius_um"] = (
        cell_slice_data["proj_cell_slice_area_um2"] / np.pi
    ) ** 0.5
    cell_slice_data.to_csv(
        cell_slice_data_file,
        header=(i == 0),
        index=False,
        mode=("w" if i == 0 else "a"),
    )

In [ ]:
cell_index_cols = [
    "sectioning_axis",
    "sectioning_axis_name",
    "section_thickness_um",
    "section_offset_um",
    "cell_id",
]
cell_slice_centroid_cols = [
    "cell_slice_centroid_x_um",
    "cell_slice_centroid_y_um",
    "cell_slice_centroid_z_um",
]
proj_cell_slice_centroid_cols = [
    "proj_cell_slice_centroid_x_um",
    "proj_cell_slice_centroid_y_um",
    "proj_cell_slice_centroid_z_um",
]
cell_slice_centroid_displacement_cols = [
    "cell_slice_centroid_displacement_x_um",
    "cell_slice_centroid_displacement_y_um",
    "cell_slice_centroid_displacement_z_um",
]
proj_cell_slice_centroid_displacement_cols = [
    "proj_cell_slice_centroid_displacement_x_um",
    "proj_cell_slice_centroid_displacement_y_um",
    "proj_cell_slice_centroid_displacement_z_um",
]
cell_slice_displacement_data = cell_slice_data.set_index(
    cell_index_cols + ["cell_slice_number"]
).sort_index()
cell_slice_displacement_data = cell_slice_displacement_data[
    cell_slice_centroid_cols + proj_cell_slice_centroid_cols
]
cell_slice_displacement_data = (
    cell_slice_displacement_data
    - cell_slice_displacement_data.groupby(cell_index_cols, sort=False).shift()
)
cell_slice_displacement_data.rename(
    columns=dict(
        zip(cell_slice_centroid_cols, cell_slice_centroid_displacement_cols)
    ),
    inplace=True,
)
cell_slice_displacement_data.rename(
    columns=dict(
        zip(proj_cell_slice_centroid_cols, proj_cell_slice_centroid_displacement_cols)
    ),
    inplace=True,
)
cell_slice_displacement_data.dropna(
    inplace=True, subset=cell_slice_centroid_displacement_cols
)
cell_slice_displacement_data.reset_index(inplace=True)
cell_slice_displacement_data.to_csv(cell_slice_displacement_data_file, index=False)